# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# # Only needed for Udacity workspace
# import importlib.util
# import sys

# # Check if 'pysqlite3' is available before importing
# if importlib.util.find_spec("pysqlite3") is not None:
#     import pysqlite3
#     sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
# TODO: Import the necessary libs
import os
import json
import chromadb
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field

# Custom library imports
from lib.agents import Agent
from lib.llm import LLM
from lib.tooling import tool
from chromadb.utils import embedding_functions

In [2]:
# TODO: Load environment variables
# load_dotenv()

OPENAI_API_KEY = "voc-11905559501688654276185689a2c33e68215.70173222"
TAVILY_API_KEY = "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"
VOCAREUM_API_BASE = "https://openai.vocareum.com/v1"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:
@tool
def retrieve_game(query: str) -> list:
    """
    Semantic search: Finds most results in the vector DB.
    args:
    - query: a question about the game industry.
    """
    # This is the stable, correct way to specify the embedding function with a custom base URL
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY,
        api_base=VOCAREUM_API_BASE,
        model_name="text-embedding-ada-002" # Ensure this matches the model used in Part 1
    )
    
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )
    
    results = collection.query(
        query_texts=[query],
        n_results=5
    )
    
    if 'documents' in results and results['documents']:
        return results['documents'][0]
    return []

#### Evaluate Retrieval Tool

In [4]:
class EvaluationReport(BaseModel):
    """Data model for the evaluation report."""
    useful: bool = Field(..., description="Whether the documents are useful to answer the question")
    description: str = Field(..., description="A detailed explanation of the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    """
    llm = LLM(
        model="gpt-4-turbo",
        temperature=0,
        api_key=OPENAI_API_KEY,
        base_url=VOCAREUM_API_BASE
    )

    prompt = f"""Your task is to evaluate if the documents are enough to respond to the query. 
    Give a detailed explanation, so it's possible to take an action to accept the information or not.
    
    Respond with ONLY a JSON object in the following format:
    {{
      "useful": <true or false>,
      "description": "<Your detailed explanation here>"
    }}

    Original user question: {question}
    Retrieved documents: {"\n\n".join(retrieved_docs)}
    """
    
    ai_message = llm.invoke(prompt)
    report_dict = json.loads(ai_message.content)
    return EvaluationReport(**report_dict)

#### Game Web Search Tool

In [5]:
@tool
def game_web_search(query: str) -> str:
    """
    Performs a web search using Tavily for questions about the game industry.
    args:
    - query: a question about the game industry. 
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    response = tavily_client.search(query=query, search_depth="advanced")
    return "\n".join([result['content'] for result in response['results']])

### Agent

In [6]:
instructions = """You are UdaPlay, an AI Research Agent for the video game industry.
Your goal is to answer user questions about video games, developers, platforms, and release dates.

Here is your workflow:
1.  **Internal Knowledge First**: Always start by using the `retrieve_game` tool.
2.  **Evaluate**: Use `evaluate_retrieval` to assess if the information is sufficient.
3.  **Web Search**: If the evaluation determines the information is not useful, use `game_web_search`.
4.  **Respond**: Combine the information into a clear and concise answer, citing your sources.
"""

udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=instructions,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    base_url=VOCAREUM_API_BASE,
    api_key=OPENAI_API_KEY)

In [7]:
queries = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]

for query in queries:
    print(f"--- Query: {query} ---\n")
    try:
        run = udaplay_agent.invoke(query)
        final_state = run.get_final_state()
        if final_state and final_state.get('messages'):
            final_answer = final_state['messages'][-1].content
            print(f"Final Answer:\n{final_answer}\n")
        else:
            print("Agent run did not produce a final answer.")
    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
    print("-" * 20)

--- Query: When was Pokémon Gold and Silver released? ---

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Final Answer:
Pokémon Gold and Silver were released in 1999. If you need more specific details, such as the exact day and month, please let me know!

--------------------
--- Query: Which one was the first 3D platformer Mario game? ---

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Final An

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes